<a href="https://colab.research.google.com/github/SangMin316/EEG_Data/blob/main/Preprocessed_MASS_S1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

공유드라이브에서 복사본을 가져온 거라서, 이름을 수정하는 작업

In [1]:
import glob
data_list = glob.glob('/content/drive/MyDrive/EEG_data/MASS/SS1/**')

In [ ]:
import os
for i in range(len(data_list)):
  # if data_list[i].split(' ')[-1] == '사본':
    print(i)
    os.rename(data_list[i], data_list[i].split('edf')[0] + 'edf')
    

data load

In [23]:
#First, investigate cassette files.
data_list = sorted(glob.glob('/content/drive/MyDrive/EEG_data/MASS/SS1/**'))
trains = [x for x in data_list if x.endswith('PSG.edf')]

labels= [x for x in data_list if x.endswith('Base.edf')]

print('number of train samples:',len(trains))
print('number of labels:',len(labels))

number of train samples: 53
number of labels: 53


In [24]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
ann = mne.read_annotations(labels[0])
print(ann.duration)
print('------')
print(ann.description)

[30. 30. 30. ... 30. 30. 30.]
------
['Sleep stage ?' 'Sleep stage ?' 'Sleep stage ?' ... 'Sleep stage ?'
 'Sleep stage ?' 'Sleep stage ?']


See imformation of EDF file 

In [25]:
!pip install pyEDFlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
from pyedflib import highlevel

signal_headers = highlevel.read_edf(trains[0])[1]

#signal_headers : list
#one signal header for each channel in the EDF.

print('number of channels:',len(signal_headers))
print('\n')

for i in range(len(signal_headers)):
    print(signal_headers[i])
    print(len(signal_headers[i]))
    print('\n')

number of channels: 31


{'label': 'EOG Left Horiz', 'dimension': 'uV', 'sample_rate': 512.0, 'sample_frequency': 512.0, 'physical_max': 730.0, 'physical_min': -730.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': '', 'transducer': ''}
10


{'label': 'EOG Right Horiz', 'dimension': 'uV', 'sample_rate': 512.0, 'sample_frequency': 512.0, 'physical_max': 728.0, 'physical_min': -728.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': '', 'transducer': ''}
10


{'label': 'EMG Chin1', 'dimension': 'uV', 'sample_rate': 512.0, 'sample_frequency': 512.0, 'physical_max': 272.0, 'physical_min': -272.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': '', 'transducer': ''}
10


{'label': 'EMG Chin2', 'dimension': 'uV', 'sample_rate': 512.0, 'sample_frequency': 512.0, 'physical_max': 267.0, 'physical_min': -267.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': '', 'transducer': ''}
10


{'label': 'EMG Chin3', 'dimension': 'uV', 'sample_rate': 512.0, 'sample_

EEG가 아닌 label 추출해서, edf 파일 읽어올 때, 제외해서 가져오기

In [27]:
exclude_channels = []
for i in range(len(signal_headers)):  
  if signal_headers[i]['label'].split(' ')[0] != 'EEG':
    exclude_channels.append(signal_headers[i]['label'])


In [28]:
print(exclude_channels)
print(len(exclude_channels))

['EOG Left Horiz', 'EOG Right Horiz', 'EMG Chin1', 'EMG Chin2', 'EMG Chin3', 'ECG ECGI', 'EMG Ant Tibial L', 'EMG Ant Tibial R', 'Resp Thermistor', 'Resp Cannula', 'Resp Belt Thor', 'Resp Belt Abdo', 'SaO2 SaO2']
13


In [29]:
import mne
data = mne.io.read_raw_edf(trains[0], exclude= exclude_channels)
X = data.get_data()

Extracting EDF parameters from /content/drive/MyDrive/EEG_data/MASS/SS1/01-01-0001 PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [30]:
X.shape

(18, 7821824)

In [32]:
signal_headers = highlevel.read_edf(trains[0])[1]

In [33]:
ann = mne.read_annotations(labels[0])
print(ann.duration)
print('------')
print(ann.description)

[30. 30. 30. ... 30. 30. 30.]
------
['Sleep stage ?' 'Sleep stage ?' 'Sleep stage ?' ... 'Sleep stage ?'
 'Sleep stage ?' 'Sleep stage ?']


In [34]:
ann.description.shape

(1018,)

In [35]:
1018*30*256

7818240

Sleep stage ? 제외하고 30초씩 잘라서 데이터 받기

In [36]:
for slice_index in range(X.shape[1] // (256 * 30)):
    # ingore the no labels
    if ann.description[slice_index] == 'Sleep stage ?':
        continue
    print(X[:, slice_index * 256 * 30: (slice_index+1) * 256 * 30].shape)
    print(ann.description[slice_index])
    if slice_index >= 40:
        break

(18, 7680)
Sleep stage W
(18, 7680)
Sleep stage W
(18, 7680)
Sleep stage W
(18, 7680)
Sleep stage W
(18, 7680)
Sleep stage W
(18, 7680)
Sleep stage 1


In [45]:
import gc
gc.collect()

44

In [ ]:
import os
import pickle
import numpy as np
for i in range(len(trains)):
    print(i)
    
    # choose EEG channels
    exclude_channels = []
    signal_headers = highlevel.read_edf(trains[i])[1]
    for j in range(len(signal_headers)):  
      if signal_headers[j]['label'].split(' ')[0] != 'EEG':
        exclude_channels.append(signal_headers[j]['label'])


    data = mne.io.read_raw_edf(trains[i],exclude = exclude_channels)
    data = data.get_data()
    data = mne.filter.resample(data, down = 1.28) # downsampling to 200Hz  
    data = mne.filter.filter_data(data, sfreq = 200, l_freq=0., h_freq=75.)
    
    # load annotaion
    ann = mne.read_annotations(labels[i])

    for slice_index in range(len(ann.description)): # 200Hz, 30epochs
    # ingore the no labels
        if ann.description[slice_index] == 'Sleep stage ?':
            continue
            
        adress = '/content/drive/MyDrive/EEG_data/MASS/SS1/Preprocessed_EEG'
        if not os.path.exists(adress+'/SS1_{}'.format(i)):
            os.makedirs(adress +'/SS1_{}'.format(i))
        
        data_adress = adress +'/SS1_{}'.format(i)
            
        data_path = data_adress +'/'+ str(i) + '_' + str(slice_index) + '.npz'
        X = data[:, slice_index * 200 * 30: (slice_index+1) * 200 * 30]
        Y = ann.description[slice_index]
        np.savez(data_path, x = X, y = Y)

In [49]:
test = glob.glob('/content/drive/MyDrive/EEG_data/MASS/SS1/Preprocessed_EEG/**')

In [50]:
len(test)

53